# Using Tensorflow to finetune for computer vision tasks

## Datasets

### Dataset Folder Structure


```bash
dataset/
├── class_1/
│   ├── image1.jpg
│   ├── image2.jpg
│   └── ...
├── class_2/
│   ├── image1.jpg
│   ├── image2.jpg
│   └── ...
└── class_n/
    ├── image1.jpg
    ├── image2.jpg
    └── ...
```

### Splitting data

In [ ]:
import tensorflow as tf

image_size = (224, 224)  
batch_size = 32          
validation_split = 0.2   

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'dataset',                  # Path to the dataset folder
    validation_split=validation_split,
    subset="training",          # Training subset
    seed=123,                   # Set a random seed for reproducibility
    image_size=image_size,      
    batch_size=batch_size       
)

# Load validation data
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'dataset',                  # Path to the dataset folder
    validation_split=validation_split,
    subset="validation",        # Validation subset
    seed=123,                   # Must be the same seed as above
    image_size=image_size,      
    batch_size=batch_size       
)

## Pre-trained Model


In [ ]:
from keras.applications import ResNet50

# Load the base model
base_model = ResNet50(
    weights='imagenet',          # Load weights pre-trained on ImageNet
    include_top=False,           # Exclude the fully connected layers
    input_shape=(224, 224, 3)    # Input size for the model
)

### Freeze model

In [ ]:
base_model.trainable = False # Freeze the base model

In [ ]:
base_model.trainable = True  # Unfreeze the base model

# Optionally, freeze some layers to fine-tune specific parts
for layer in base_model.layers[:100]:
    layer.trainable = False

## Custom layers

In [ ]:
from keras import layers, models


# Add custom layers
model = models.Sequential([
    base_model,                             # Pre-trained base
    layers.GlobalAveragePooling2D(),       # Pooling layer to reduce dimensions
    layers.Dense(128, activation='relu'),  # Fully connected layer
    layers.Dropout(0.5),                   # Dropout for regularization
    layers.Dense(10, activation='softmax') # Final layer for classification
])

## Training

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)
model.summary() # Display the model's architecture

# Special: Image + metadata model

In [ ]:
from keras.applications import ResNet50
from keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, Concatenate
from keras.models import Model

image_input = Input(shape=(224, 224, 3), name="image_input")
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=image_input)
base_model.trainable = False  

image_features = base_model.output
image_features = GlobalAveragePooling2D()(image_features)
image_features = Dense(128, activation='relu')(image_features)
image_features = Dropout(0.5)(image_features)

metadata_input = Input(shape=(10,), name="metadata_input")  
metadata_features = Dense(64, activation='relu')(metadata_input)
metadata_features = Dropout(0.3)(metadata_features)
metadata_features = Dense(32, activation='relu')(metadata_features)

In [ ]:
combined_features = Concatenate()([image_features, metadata_features])

x = Dense(128, activation='relu')(combined_features)
x = Dropout(0.5)(x)
output = Dense(10, activation='softmax', name="output_layer")(x) 


In [ ]:
# Define the multi-input model
model = Model(inputs=[image_input, metadata_input], outputs=output)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()